In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

print(tf.__version__)

In [1]:
import numpy as np
# from sklearn.model_selection import train_test_split

datasetFileName = "dataset.npz"
with np.load(datasetFileName) as data:
    dataImages = data['images']
    dataLabels = data['labels']

classNames = ['Clean', 'Contaminated']


In [2]:
print(dataImages.shape)
dataImages.dtype


(248, 326, 244, 3)


dtype('uint8')

In [ ]:
# dataShape = (len(dataImages),)+dataImages[0].shape
# print(dataShape)

In [ ]:
# a = np.empty(dataShape, dtype=np.uint8)

# for i in range(len(dataImages)):
#     a[i] = dataImages[i].astype('uint8')

# dataImages = a

# del a

In [ ]:
print(dataLabels.shape)
dataLabels.dtype


In [ ]:
# dataLabels = np.uint8(dataLabels)

# print(dataLabels.shape)
# dataLabels.dtype

In [ ]:

# Split the data up in train and test sets
trainImages, testImages, trainLabels, testLabels = train_test_split(dataImages, dataLabels, test_size=0.33, random_state=42)


In [ ]:
trainImages.shape

In [ ]:
trainImages.dtype

In [ ]:
trainImages[0]

In [ ]:
len(trainLabels)

In [ ]:
trainLabels

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.imshow(trainImages[0])
plt.xlabel(classNames[trainLabels[0]])
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
trainImages = trainImages / 255.0

testImages = testImages / 255.0

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(trainImages[i], cmap=plt.cm.binary)
    plt.xlabel(classNames[trainLabels[i]])
plt.show()

In [ ]:
inputShape = trainImages[0].shape
print(inputShape)
outputShape = len(classNames)
print(outputShape)

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    # tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Flatten(input_shape=inputShape),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(outputShape)
])
model.summary()


In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.fit(trainImages, trainLabels, epochs=10)

In [ ]:
testLoss, testAcc = model.evaluate(testImages,  testLabels, verbose=2)

print('\nTest accuracy:', testAcc)

In [ ]:
predictions = model.predict(testImages)
predictions[0]

In [ ]:
np.argmax(predictions[0])

In [ ]:
testLabels[0]

In [ ]:
probabilityModel = tf.keras.Sequential([model, tf.keras.layers.Softmax()])


In [ ]:
predictions = probabilityModel.predict(testImages)


In [ ]:
predictions[0]

In [ ]:
np.argmax(predictions[0])

In [ ]:
testLabels[0]

In [ ]:
import matplotlib.pyplot as plt

def plot_image(i, predictionsArray, trueLabels, img):
  trueLabel, img = trueLabels[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predictedLabel = np.argmax(predictionsArray)
  if predictedLabel == trueLabel:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(classNames[predictedLabel],
                                100*np.max(predictionsArray),
                                classNames[trueLabel]),
                                color=color)

def plot_value_array(i, predictionsArray, trueLabels):
  trueLabel = trueLabels[i]
  plt.grid(False)
  plt.xticks(range(outputShape))
  plt.yticks([])
  thisplot = plt.bar(range(outputShape), predictionsArray, color="#777777")
  plt.ylim([0, 1])
  predictedLabel = np.argmax(predictionsArray)

  thisplot[predictedLabel].set_color('red')
  thisplot[trueLabel].set_color('blue')

In [ ]:
import matplotlib.pyplot as plt

i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], testLabels, testImages)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  testLabels)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], testLabels, testImages)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  testLabels)
plt.show()

In [ ]:
testLabels[i]

In [ ]:
import matplotlib.pyplot as plt

# Plot the first X test images, their predicted labels, and the true labels.
# Color correct predictions in blue and incorrect predictions in red.
high = testLabels.shape[0]
numRows = 5
numCols = 3
numImages = numRows*numCols

selectedIndex = np.random.randint(0, high, size=numImages)

somePredictions = predictions[selectedIndex]
someImages = testImages[selectedIndex]
someLabels = testLabels[selectedIndex]

plt.figure(figsize=(2*2*numCols, 2*numRows))
for i in range(numImages):
    plt.subplot(numRows, 2*numCols, 2*i+1)
    plot_image(i, predictions[i], someLabels, someImages)
    plt.subplot(numRows, 2*numCols, 2*i+2)
    plot_value_array(i, predictions[i], testLabels)
plt.tight_layout()
plt.show()

In [ ]:
# Grab an image from the test dataset.
img = testImages[1]

print(img.shape)

In [ ]:
import numpy as np

# Add the image to a batch where it's the only member.
img = (np.expand_dims(img,0))

print(img.shape)

In [ ]:
predictionsSingle = probabilityModel.predict(img)

print(predictionsSingle)

In [ ]:
plot_value_array(1, predictionsSingle[0], testLabels)
_ = plt.xticks(range(outputShape), classNames, rotation=45)
plt.show()

In [ ]:
np.argmax(predictionsSingle[0])

In [ ]:
print(testAcc)

version = f'{testAcc*100:.0f}'

print(version)



In [ ]:
import os

exportPath = ""

if testAcc > 0.95:
    modelDir = 'tf_model'
    version = f"4_{(testAcc*100):.0f}"

    # for SavedModel format
    exportPath = os.path.join(modelDir, version)
    # save the model
    model.save(exportPath, save_format="tf")

    print('\nexport path = {}'.format(exportPath))

    !dir {exportPath}

In [ ]:
x_img = testImages[5]

plt.imshow(x_img)
plt.xlabel(classNames[testLabels[5]])
plt.show()

In [ ]:
# load SavedModel format

newModel = tf.keras.models.load_model(exportPath)

# Check its architecture
newModel.summary()

In [ ]:
# Evaluate the restored model
loss, acc = newModel.evaluate(testImages, testLabels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

newPredictions = newModel.predict(testImages)

print(newPredictions.shape)

In [ ]:
print("prediction: ", np.argmax(newPredictions[0]))
print("actual: ", testLabels[0])


In [ ]:
testLabels[0]

In [ ]:
import json
# create json object

jsonData = json.dumps({"signature_name":"serving_default", "instances":testImages[34].tolist()})

In [ ]:
import requests

apiRunning=False

if apiRunning == True:
    headers = {"content-type":"application/json"}
    json_response = requests.post('http://localhost:8501/v1/models/giwa:predict', data=jsonData, headers=headers)

    predictions = json.loads(json_response.text)['predictions']

    pred = [np.argmax(predictions[p]) for p in range(len(predictions))]
    print("Predictions: ", pred)
    print("Actual: ", testLabels[5].tolist())

In [ ]:
#  HDF5 format

import os

exportPath = ""

if testAcc > 0.95:
    modelDir = 'tf_model'
    version = f"4_{(testAcc*100):.0f}"

    # for HDF5 format
    exportPath = os.path.join(modelDir, f"{version}.h5")
    # Save the entire model to a HDF5 file.
    # The '.h5' extension indicates that the model should be saved to HDF5.
    model.save(exportPath)

    !dir {exportPath}

In [ ]:

# Recreate the exact same model, including its weights and the optimizer
newModelHF = tf.keras.models.load_model(exportPath)

# Show the model architecture
newModelHF.summary()

In [ ]:
# Evaluate the restored model
loss, acc = newModelHF.evaluate(testImages, testLabels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

newPredictions = newModelHF.predict(testImages)

print(newPredictions.shape)

In [ ]:
print("prediction: ", np.argmax(newPredictions[0]))
print("actual: ", testLabels[0])
